<p><font size="6" color='grey'> <b>
Machine Learning
</b></font> </br></p>
<p><font size="5" color='grey'> <b>
Autoencoder - Network-Intrusion - Detektor
</b></font> </br></p>

---


# 0  | Install & Import
***

In [ ]:
# Install

In [ ]:
# Import
from pandas import read_csv, DataFrame, concat
from numpy import power, quantile, mean
import numpy as np

from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
    roc_auc_score
)

import keras
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import regularizers
from keras.utils import set_random_seed, plot_model

from tensorflow import keras
from tensorflow.config.experimental import enable_op_determinism
import tensorflow as tf

import plotly.express as px
import plotly.subplots as sp

import matplotlib.pyplot as plt

In [ ]:
# Warnung ausstellen
import warnings
warnings.filterwarnings("ignore")

# 1  | Understand
***


<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Aufgabe verstehen</br>
✅ Daten sammeln</br>
✅ Statistische Analyse (Min, Max, Mean, Korrelation, ...)</br>
✅ Datenvisualisierung (Streudiagramm, Box-Plot, ...)</br>
✅ Prepare Schritte festlegen</br>

<p><font color='black' size="5">
Anwendungsfall
</font></p>

Dies ist der Datensatz, der für den Third International Knowledge Discovery and Data Mining Tools Competition verwendet wurde. Die Wettbewerbsaufgabe bestand darin, einen Netzwerk-Intrusion-Detektor zu bauen, ein Vorhersagemodell, das in der Lage ist, zwischen schlechten Verbindungen, sogenannten Intrusionen oder Angriffen, und guten normalen Verbindungen zu unterscheiden. Diese Datenbank enthält einen zu prüfenden Standarddatensatz, der eine Vielzahl von Eindringversuchen umfasst, die in einer militärischen Netzwerkumgebung simuliert wurden.

Der ursprüngliche KDD Cup 1999-Datensatz aus dem  UCI-Repositorium für maschinelles Lernen  enthält 41 Attribute (34 kontinuierlich und 7 kategorial), sie werden jedoch auf 3 Attribute (Dauer, src_bytes, dst_bytes) reduziert.

Da die kontinuierlichen Attributwerte um '0' herum konzentriert sind, haben wir jeden Wert durch y = log(x + 0,1) in einen Wert weit von '0' transformiert.

Aus KDD Cup 1999-Datensatz werden ca. 500k „http“-Dienstdaten verwendet.

Der KDD Cup ist der jährliche Data-Mining- und Knowledge-Discovery-Wettbewerb, der von der ACM Special Interest Group on Knowledge Discovery and Data Mining, der führenden Berufsorganisation von Data-Minern, organisiert wird.


[DataSet](https://www.openml.org/search?type=data&status=active&id=1113)

[Info](http://odds.cs.stonybrook.edu/http-kddcup99-dataset/)


[UCI](https://archive.ics.uci.edu/ml/datasets/kdd+cup+1999+data)

[KDD](https://kdd.org/kdd-cup)



**Datensatz:**

| feature name  | description                                      | type        |
|------------|--------------------------------------------------|-------------|
| duration   | length (number of seconds) of the connection     | continuous  |
| src_bytes  | number of data bytes from source to destination  | continuous  |
| dst_bytes  | number of data bytes from destination to source  | continuous  |

<br>
<br>


In [ ]:
df = read_csv(
    "https://raw.githubusercontent.com/ralf-42/ML_Intro/main/02_daten/05_tabellen/kddcup1999_xs.csv"
)


<p><font color='black' size="5">
Trennung Data/Target
</font></p>

In [ ]:
data = df.copy()
target = data.pop("Intrusion")

In [ ]:
target.value_counts()


<p><font color='black' size="5">
Trennung Inlier/Outlier
</font></p>

In [ ]:
inlier_data = df[df.Intrusion == 0].copy()
inlier_target = inlier_data.pop("Intrusion")

In [ ]:
outlier_data = df[df.Intrusion == 1].copy()
outlier_target = outlier_data.pop("Intrusion")

# 2 | Prepare

---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Nicht benötigte Features löschen</br>
✅ Datentyp ermitteln/ändern</br>
✅ Duplikate ermitteln/löschen</br>
✅ Missing Values behandeln</br>
✅ Ausreißer behandeln</br>
✅ Kategorischer Features Kodieren</br>
✅ Numerischer Features skalieren</br>
✅ Feature-Engineering (neue Features schaffen)</br>
✅ Dimensionalität reduzieren</br>
✅ Resampling (Over-/Undersampling)</br>
✅ Pipeline erstellen/konfigurieren</br>
✅ Train-Test-Split durchführen</br>

# 3 | Modeling
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellauswahl treffen</br>
✅ Pipeline erweitern/konfigurieren</br>
✅ Training durchführen</br>
✅ Hyperparameter Tuning</br>
✅ Cross-Valdiation</br>
✅ Bootstrapping</br>
✅ Regularization</br>

 <p><font color='black' size="5">
Zufallszahl initialisieren
</font></p>

In [ ]:
set_random_seed(42)
enable_op_determinism()


<p><font color='black' size="5">
Modellaufbau
</font></p>

In [ ]:
input_dim = inlier_data.shape[1]

**Keras Functional API**

Functional API ermöglicht, Modelle mit komplexeren Topologien wie mehreren Eingängen und Ausgängen oder Modellen mit gemeinsam genutzten Schichten zu erstellen. Die funktionale API wird verwendet, um direkte Verbindungen zwischen den Schichten explizit zu definieren, was mehr Kontrolle und Anpassungsfähigkeit bietet:

In [ ]:
# Erstellen des Input-Layers
input_layer = Input(shape=(input_dim,), name='Input_layer')

In [ ]:
# Erstellen des Encoder-Layers
encoder_layer = Dense(
    units=10,
    activation="relu",
    activity_regularizer=regularizers.l1(10e-5),
    name='Encoder'
)(input_layer)

In [ ]:
# Erstellen des Decoder-Layers
decoder_layer = Dense(
    units=input_dim,
    activation="sigmoid",
    name='Decoder'
)(encoder_layer)

In [ ]:
# Zusammenbau des Autoencoders
autoencoder = Model(inputs=input_layer, outputs=decoder_layer)

In [ ]:
autoencoder.summary()

In [ ]:
# Visualisierung neuronales Netz
plot_model(
    autoencoder,
    to_file="nn_structure.png",
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True,
    dpi=100,
    expand_nested=True,
    show_layer_activations=True
)

In [ ]:
# Anzahl Parameter je Layer
for layer in autoencoder.layers:
    print(f"{layer.name}: {layer.count_params()} Parameter")


<p><font color='black' size="5">
 Compile
</font></p>

In [ ]:
autoencoder.compile(optimizer="adam", loss="mae")

<p><font color='black' size="5">
Callbacks
</font></p>

In [ ]:
early = EarlyStopping(monitor="val_loss", patience=2)

check = ModelCheckpoint(filepath="model.keras", monitor="val_loss", save_best_only=True)


<p><font color='black' size="5">
 Training
</font></p>

In [ ]:
autoencoder.fit(
    inlier_data,
    inlier_data,
    epochs=50,
    batch_size=512,
    shuffle=True,
    validation_split=0.2,
    callbacks=[early, check],
)


<p><font color='black' size="5">
Rekonstruktion
</font></p>

In [ ]:
model = load_model("model.keras")

In [ ]:
# Daten durch den trainierten Autoencoder laufen lassen, um die rekonstruierten Daten zu erhalten
reconstructed_data = model.predict(data)


<p><font color='black' size="5">
Rekonstruktionsfehler berechnen
</font></p>

In [ ]:
# Berechnen des mittleren quadratischen Fehlers (MAE) zwischen den Originaldaten und den rekonstruierten Daten
# Dies wird für jeden Datenpunkt einzeln durchgeführt, indem die Differenz quadriert und dann gemittelt wird.
# 'axis=1' bedeutet, dass die Operation entlang der Spalten für jede Zeile durchgeführt wird, also für jeden Datenpunkt.
mae = mean(np.power(data - reconstructed_data, 2), axis=1)

# Bestimmen des Schwellenwerts für den MAE, der verwendet wird, um Ausreißer zu identifizieren
# Hier wird der 99.5%-Quantilwert des MAE verwendet, was bedeutet, dass 99.5% der Datenpunkte einen MAE haben,
# der niedriger ist als dieser Schwellenwert. Die Annahme hier ist, dass die oberen 0.5% der Datenpunkte mit dem
# höchsten MAE als Ausreißer betrachtet werden.
mae_threshold = quantile(mae, 0.995)  # Annahme: 0.5% der Daten sind Ausreißer

# Identifizieren der Ausreißer als die Datenpunkte, deren mae den berechneten Schwellenwert überschreitet
# Dies gibt ein boolesches Array zurück, wobei 'True' einen Ausreißer (mae über dem Schwellenwert) darstellt
# und 'False' einen normalen Datenpunkt (mae unter dem Schwellenwert).
outliers_detected = mae > mae_threshold

In [ ]:
DataFrame(outliers_detected).value_counts()

In [ ]:
plt.hist(mae, bins=50)
plt.axvline(mae_threshold, color="r", linestyle="dashed", linewidth=2)
plt.title("Histogramm der Rekonstruktionsfehler")
plt.xlabel("Rekonstruktionsfehler")
plt.ylabel("Anzahl der Datenpunkte")
plt.show()

# 4 | Evaluate
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Prognose (Train, Test) erstellen</br>
✅ Modellgüte prüfen</br>
✅ Residuenanalyse erstellen</br>
✅ Feature Importance/Selektion prüfen</br>
✅ Robustheitstest erstellen</br>
✅ Modellinterpretation erstellen</br>
✅ Sensitivitätsanalyse erstellen</br>
✅ Kommunikation (Key Takeaways)</br>

<p><font color='black' size="5">
Silhouette Koeffizient
</font></p>

In [ ]:
labels = outliers_detected.replace([False, True], [0, 1])


<p><font color='black' size="5">
 ROC AUC
</font></p>

In [ ]:
roc_auc_score(target, labels)


<p><font color='black' size="5">
Confusion Matrix
</font></p>

In [ ]:
conf_matrix = confusion_matrix(target, labels)
plt.rcParams["figure.figsize"] = [5, 5]
disp = ConfusionMatrixDisplay(conf_matrix, display_labels=["No", "Yes"])
disp.plot(cmap="Blues")

In [ ]:
print(classification_report(target, labels, target_names=["No", "Yes"]))

<p><font color='black' size="5">
Aufbau Analysewürfel
</font></p>

In [ ]:
# Übernahme der Testdaten
cube = data.copy()

# Übernahem Target real & predict
cube["real"] = target
cube["predict"] = labels

In [ ]:
# Erstellung 2D Features über Dimensionsreduktion PCA - unsupervised
from sklearn.decomposition import PCA

pca = PCA()
pca = pca.fit_transform(data)
pca_df = DataFrame(pca)

cube.reset_index(inplace=True)

In [ ]:
# Cube um pca erweitern
cube["PCA1"] = pca_df[0]
cube["PCA2"] = pca_df[1]

<p><font color='black' size="5">
Visualisierung real vs predict
</font></p>

In [ ]:
# Histogramm
title_ = "Histogramm real vs predict"
fig = px.histogram(cube, x=["real", "predict"], nbins=2, text_auto=".2s", title=title_)
fig.update_layout(barmode="group", bargap=0.1, width=600, height=600)
fig.show()

In [ ]:
# 2 x Scatterplots

cube["real_cat"] = cube["real"].astype(str)
cube["predict_cat"] = cube["predict"].astype(str)

title_ = "Streupunktdiagramm real"
img1 = px.scatter(
    cube, x="PCA1", y="PCA2", color="real_cat", opacity=0.5, width=600, height=600
)

title_ = "Streupunktdiagramm predict"
img2 = px.scatter(
    cube, x="PCA1", y="PCA2", color="predict_cat", opacity=0.5, width=600, height=600
)

fig = sp.make_subplots(
    rows=1, cols=2, subplot_titles=("Scatterplot real", "Scatterplot predict")
)

for trace in img1.data:
    fig.add_trace(trace, row=1, col=1)
for trace in img2.data:
    fig.add_trace(trace, row=1, col=2)

# Layout anpassen
fig.update_layout(width=1000, height=500, title_text=title_)

# Plot anzeigen
fig.show()

In [ ]:
# real <> predict
cube[cube.real != cube.predict].describe().T

In [ ]:
cube[cube.real != cube.predict]

# 5 | Deploy
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellexport und -speicherung</br>
✅ Abhängigkeiten und Umgebung</br>
✅ Sicherheit und Datenschutz</br>
✅ In die Produktion integrieren</br>
✅ Tests und Validierung</br>
✅ Dokumentation & Wartung</br>